# Loading Related Modules

In [2]:
import json
import logging
from pprint import pprint
from time import sleep
import re


In [3]:
import requests
from lxml import html
from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch


# Scraping Content from Webpage  

In [4]:
url = 'https://www.cbinsights.com/research-unicorn-companies'
req = requests.get(url)
page = req.text
soup = BeautifulSoup(page, 'lxml')
#print soup.prettify()

In [5]:
"""Select data table content"""
table = soup.select('.sortable-theme-bootstrap')[0]

# Parsing Related Table Content

In [6]:
def parse_table(table):
    """ Get data from table """
    
    parsed = []
    for row in table.find_all("tr"):
        for cell in row.find_all(['td','th']):
            parsed.append(cell.get_text())
            
    header = parsed[:6]
    result = []
    
    """convert list mappings into dictionary """
    
    for row in [parsed[i:i + 6] for i in range(6, len(parsed), 6)]:
        temp ={}
        for n in range(6):
            ## data cleaning : remove extra characters /n/t, $symbol
            row[n] = re.sub('\s+', ' ', row[n])
            row[n] =row[n].replace("$", "")

            temp[header[n]]=row[n]
            
        result.append(temp)        
            
    return result

In [7]:
unicorn = parse_table(table)

In [8]:
# Check top 5 unicorn company info

unicorn[:5]

[{u'Company': u'Uber',
  u'Country': u'United States',
  u'Date Joined': u'8/23/2013',
  u'Industry': u'On-Demand',
  u'Select Investors': u'Lowercase Capital, Benchmark Capital, Google Ventures',
  u'Valuation ($B)': u'72'},
 {u'Company': u'Didi Chuxing ',
  u'Country': u'China',
  u'Date Joined': u'12/31/2014',
  u'Industry': u'On-Demand',
  u'Select Investors': u'Matrix Partners, Tiger Global Management, Softbank Corp.,',
  u'Valuation ($B)': u'56'},
 {u'Company': u'Airbnb',
  u'Country': u'United States',
  u'Date Joined': u'7/26/2011',
  u'Industry': u'eCommerce/Marketplace',
  u'Select Investors': u'General Catalyst Partners, Andreessen Horowitz, ENIAC Ventures',
  u'Valuation ($B)': u'29.3'},
 {u'Company': u'SpaceX',
  u'Country': u'United States',
  u'Date Joined': u'12/1/2012',
  u'Industry': u'Other Transportation',
  u'Select Investors': u'Founders Fund, Draper Fisher Jurvetson, Rothenberg Ventures',
  u'Valuation ($B)': u'21.5'},
 {u'Company': u'Palantir Technologies',
  u'

# Create Unicorn Index

In [9]:
def create_index(es_object, index_name):
    created = False
    # index settings
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
    "mappings": {
    "default":{
      "dynamic": "strict",
      "properties": {
       "Company":{
          "type": "text"
        },
        
        "Country":{
          "type": "text",
          "fields": {
            "keyword":{
              "type": "keyword"
            }
          }
        },
        
        "Date Joined":{
          "type": "date",
          "format": "MM/dd/yyyy"
        },
        "Industry":{
          "type": "text",
          "fields": {
            "keyword":{
              "type": "keyword"
            }
          }                  
        },
        "Select Investors":{
          "type": "text"
        },
        "Valuation ($B)": {
          "type": "float"
        }
      }
    }
  }
}


    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [10]:
def store_record(elastic_object, index_name, idx, record):
    is_stored = True
    try:
        outcome = elastic_object.index(index=index_name, doc_type='default', id= idx, body=record)
        # un-pound the following line to check indexing result
        # print(outcome['result'])
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))
        is_stored = False
    finally:
        return is_stored

In [11]:
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Yay Connected')
    else:
        print('Awww it could not connect!')
    return _es

In [12]:
def search(es_object, index_name, search ):
    res = es_object.search(index = index_name, body = search)
    pprint(res)

# Add document in a specific index

In [13]:
es = connect_elasticsearch()

for idx, row in enumerate(unicorn):
    if es is not None:
        if create_index(es, 'unicorn'):
            out = store_record(es, 'unicorn', idx, row)
            #print('Data indexed successfully')

Yay Connected


# Execute search requests

In [14]:
es = connect_elasticsearch()

search_object_1 = {
  "query":{
    "match": {
    "Select Investor": "Softbank"
    }
  }
}

search_object_2 ={
  "size": 0,
  "aggs": {
    "number_of_companies": {
      "terms": {
        "field": "Country.keyword"
         }
        }
      }
    }
  
search_object_3 = {
  "size": 0,
  "aggs": {
    "valuation_dist": {
      "histogram": {
        "field": "Valuation ($B)",
        "interval": 5,
        "min_doc_count": 1
      }
    }
  }
}
search(es, 'unicorn', search_object_2)

Yay Connected
{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'aggregations': {u'number_of_companies': {u'buckets': [{u'doc_count': 125,
                                                          u'key': u'United States'},
                                                         {u'doc_count': 73,
                                                          u'key': u'China'},
                                                         {u'doc_count': 14,
                                                          u'key': u'India'},
                                                         {u'doc_count': 12,
                                                          u'key': u'United Kingdom'},
                                                         {u'doc_count': 6,
                                                          u'key': u'Germany'},
                                                         {u'doc_count': 4,
                                                    

# Optional: Perform Queries through Kibana DevTool Console

### Create json file for bulk import

In [15]:
"""In order to create json file for bulk import, we shall add action data for each index"""

action_list = []
for i in range(len(unicorn)):
    # create index for each entry
    action = { "index" : { "_index" : "unicorn", "_type" : "default", "_id" : i+1 } }
    action_list.append(action)

In [16]:
"""create json file in specific format for bulk API"""
## action_and_meta_data\n
## optional_source\n

with open('unicorn.json', 'w') as outfile:
    for (idx, line) in zip(action_list, unicorn):
        json.dump(idx, outfile)
        outfile.write('\n')
        json.dump(line, outfile)
        outfile.write('\n')

### Create index and mappings

```
PUT /unicorn

{
  "mappings":{   
  
    "default":{
      "properties": {
        "Company":{
          "type": "text"
        },
        
        "Country":{
          "type": "text",
          "fields": {
            "keyword":{
              "type": "keyword"
            }
          }
        },
        
        "Date Joined":{
          "type": "date",
          "format": "MM/dd/yyyy"
        },
        "Industry":{
          "type": "text",
        },
        "Select Investors":{
          "type": "text",
          "fields": {
            "keyword":{
              "type": "text"
            }
          }
        },
        "Valuation ($B)": {
          "type": "float"
        }
      }
    }
  }
}
```


### Index documents via bulk

``` 
$ curl -s -H "Content-Type: application/xOST localhost:9200/unicorn/_bulk --data-binary "@unicorn.json" 
```

### Query Examples 

_we can use Kibana Devtool to do the query_ 
_How many unicorn company was founded during year 2017?_

```
GET /unicorn/default/_search

{
  "size": 0,
  "aggs": {
    "date_range": {
      "range": {
        "field": "Date Joined",
        "ranges": [
          {
            "to": "01/01/2017||+1y",
            "from": "01/01/2017"
            
          }
        ]
      }
    }
  }
}
```

__Returning Result__

```
{
  "took": 15,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": 260,
    "max_score": 0,
    "hits": []
  },
  "aggregations": {
    "date_range": {
      "buckets": [
        {
          "key": "01/01/2017-01/01/2018",
          "from": 1483228800000,
          "from_as_string": "01/01/2017",
          "to": 1514764800000,
          "to_as_string": "01/01/2018",
          "doc_count": 66
        }
      ]
    }
  }
}
```